# Data Preparation

In [1]:
import numpy as np
import pandas as pd

In [2]:
gens = pd.read_csv('data/gens.csv').amount.values.reshape(-1, 1)  # ~2023. 10. 16. 00:00
pred = pd.read_csv('data/pred.csv')
pred = pred.loc[pred['round']==1, :].amount  # ~2023. 10. 16. 00:00
pred = pred.values.reshape(-1, 5)
errors = np.abs(gens-pred) / 99.0 * 100

In [3]:
errors_after = pd.read_csv('data/bid_results.csv').sort_values(by=['time', 'Model']).reset_index(drop=True)  # 10. 17. 01:00~10. 27. 00:00
display(errors_after.time[-120:])
errors_after = errors_after.error.values
errors_after = errors_after.reshape(-1, 5)
errors_test = errors_after[-24:, :].copy()  # 10. 26. 01:00~10. 27. 00:00 (24 hours)
errors_after = errors_after[:-24, :]  # ~10. 26. 00:00

1080    2023-10-26T01:00:00+09:00
1081    2023-10-26T01:00:00+09:00
1082    2023-10-26T01:00:00+09:00
1083    2023-10-26T01:00:00+09:00
1084    2023-10-26T01:00:00+09:00
                  ...            
1195    2023-10-27T00:00:00+09:00
1196    2023-10-27T00:00:00+09:00
1197    2023-10-27T00:00:00+09:00
1198    2023-10-27T00:00:00+09:00
1199    2023-10-27T00:00:00+09:00
Name: time, Length: 120, dtype: object

In [4]:
errors = np.concatenate([errors, errors_after], axis=0)
del errors_after
errors.shape

(11832, 5)

# Modeling

## Ready

In [5]:
import torch
from torch import nn, FloatTensor
device = torch.device('cuda')
from tools.DLinear import Model as Dlinear
from tools.RegressionTrainingTools import Trainer
from torch.utils.data import Dataset, DataLoader

In [6]:
class CustomDataset(Dataset):
    def __init__(self, data, window_size, forecast_size):
        super().__init__()

        self.x = []; x_append = self.x.append
        self.y = []; y_append = self.y.append
        self.length = data.shape[0] - window_size - forecast_size + 1

        for i in range(self.length):
            x_append(FloatTensor(data[i:i+window_size, :]))
            y_append(FloatTensor(data[i+window_size:i+window_size+forecast_size, :]))
    
        return
    

    def __len__(self):
        return self.length
    

    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [7]:
train_dataset = CustomDataset(errors[:-96, :], window_size=72, forecast_size=48)
val_dataset = CustomDataset(errors[-96-72-48+1:, :], window_size=72, forecast_size=48)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)

errors_test_x = torch.FloatTensor(errors[-96:, :]).reshape(1, 96, 5).to(device)

In [8]:
class Config:
    def __init__(self, seq_len=10, pred_len=2, individual=True, enc_in=10):
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.individual = individual
        self.enc_in = enc_in  # channels
        return

configs = Config(seq_len=72, pred_len=48, individual=True, enc_in=5)
model = Dlinear(configs).to(device)

## Training Process

In [9]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

trainer = Trainer(criterion, device, save_path='checkpoints/errors_to_errors_1.pt')
best_model = trainer.train(model, optimizer, train_loader, val_loader, patience=10, epochs=100)

Epoch 67: 100%|██████████| 364/364 [00:01<00:00, 268.18it/s, Train Loss=57.1774, Valid Loss=28.6386, Valid R2=0.0000, Valid MAE=3.2755]


Early Stopped
